# Pulse measurement
This script is used to apply a voltage pulse with defined parameters and capture the applied pulse as well as device response at the oscilloscope

#### Import library files and list instruments connected to the resource manager

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyvisa
from datetime import datetime
from scr import *

rm = pyvisa.ResourceManager()
print(rm.list_resources()) # list connected instruments

('TCPIP0::192.168.0.5::inst0::INSTR', 'GPIB0::10::INSTR')


#### Open a connection with resource manager

In [29]:
pulsegen = rm.open_resource("GPIB0::10::INSTR")
scope = rm.open_resource("TCPIP0::192.168.0.5::inst0::INSTR")
print(pulsegen.query("*IDN?"))
print(scope.query("*IDN?"))

HEWLETT-PACKARD,HP81110A,DE38700131,REV 01.12.03

AGILENT TECHNOLOGIES,DSO-X 3034A,MY53160262,02.43.2018020635



#### Initialize the instruments

In [30]:
# ======== initialize pulse generator ====================

pulsegen.write("*RST")
pulsegen.write(":DISPlay On")
pulsegen.write(":HOLD VOLT") # enable voltage or current subsystem
pulsegen.write(":ROSC:SOUR Int")  # external reference clock
pulsegen.write(":OUTP1:IMP 50OHM")  # set OUT1 source impedance
pulsegen.write(":OUTP1:IMP:EXT 50OHM")  # or 1MOHM 
pulsegen.write(":OUTP1:POL NORM")   # set OUT1 polarity
pulsegen.write(":CHAN:MATH OFF")  # channel math off
pulsegen.write(":TRIG:COUN 1")
pulsegen.write(":ARM:SOUR MAN")
#pulsegen.write("*WAI")

# ======== initialize Oscilloscope =======================

scope.write("*CLS")
scope.write("*RST")

scope.write("CHAN1:DISP OFF")  # enable channel 1 trace
scope.write("CHAN1:COUP DC")  # channel 1 coupling
scope.write("CHAN1:BWL OFF") # set channel 1 bandwidth to full bandwidth of oscilloscope
scope.write("CHAN1:IMP FIFT") # or ONEMeg
scope.write("CHAN1:SCAL 1")  # set channel 1 vertical scale
scope.write("CHAN1:OFFS 0")  # set channel 1 offset

scope.write("CHAN2:DISP ON")  
scope.write("CHAN2:COUP DC")  
scope.write("CHAN2:BWL OFF") 
scope.write("CHAN2:IMP FIFT") #FIFT
scope.write("CHAN2:SCAL 1")  
scope.write("CHAN2:OFFS 0")  

scope.write(":TIMebase:SCALe 1")
scope.write(":TIMebase:POSition 0")

scope.write("TRIG:SOUR EXT") # Externally triggered from pulse generator
scope.write("TRIG:MODE EDGE") # Edge triggered
scope.write("TRIG:SLOP POS") # Triggered on positive slope



15

#### Set the pulse parameters, apply pulse and aquire data

In [31]:
# ========= Set pulse parameters =============
ep = PulseParameters(
    amplitude=5,  # V
    pulsewidth=100,  # ns
    risetime=2,  # ns
    falltime=2,  # ns
)
Rs = 350
pulsegen.write(f":VOLT1 {ep.amplitude}V")
pulsegen.write(f":VOLT1:OFFS {ep.amplitude/2}V") # set offset = amplitude/2
pulsegen.write(f":PULS:WIDT {ep.pulsewidth}NS")
pulsegen.write(f":PULS:TRAN {ep.risetime}NS") # leading edge transition time
pulsegen.write(f":PULS:TRAN:TRA:AUTO OFF") # trailing edge = leading edge OFF
pulsegen.write(f":PULS:TRAN:TRA {ep.falltime}NS") # trailing edge transition time

# ===== Set Recording parameters at scope =======
Vout = ep.amplitude * (50/ (50 + Rs))
scope.write(f"CHAN2:SCAL {Vout/4/1.5}")
scope.write(f"CHAN2:OFFS {Vout/2/1.5}")
pulseLength = (ep.pulsewidth + ep.risetime + ep.falltime) * 1e-9
scope.write(f"TIM:SCAL {pulseLength/4}")
scope.write(f"TIM:POS {pulseLength}")
#scope.write("TRIG:LEV 1")
scope.write(":ACQUIRE:TYPE HRES")
#scope.write(":ACQUIRE:COUNT 5") # used for averaging of multiple aquisitions
scope.write(":SINGLE") # Acquire a single trigger of data

# ====== Trigger the pulse =====================
pulsegen.write(":OUTP1 ON")  # Turn on OUT1

sleep(0.5)
pulsegen.write("*TRG")
sleep(1)
pulsegen.write(":OUTP1 OFF")  # Turn off OUT1


# ========= Aquire data =======================
waveform = WaveForm()
# store all this information into a waveform structure for later use
#scope.write(f":WAVEFORM:SOURCE CHAN1")
# Get the data back as a WORD(i.e., INT16), other options are ASCII and BYTE
#scope.write(":WAVEFORM:FORMAT WORD")
# Set the byte order on the instrument as well
#scope.write(":WAVEFORM:BYTEORDER LSBFirst")
# Get the preamble block
#preambleBlock = scope.query(":WAVeform:PREamble?")
#rawData = scope.query_binary_values(":WAV:DATA?", "H")
#waveform.addChannel(1, preambleBlock.split(","), rawData)

scope.write(f":WAVEFORM:SOURCE CHAN2")
scope.write(":WAVEFORM:FORMAT WORD")
scope.write(":WAVEFORM:BYTEORDER LSBFirst")
preambleBlock = scope.query(":WAVeform:PREamble?")
rawData = scope.query_binary_values(":WAV:DATA?", "H")
waveform.addChannel(2, preambleBlock.split(","), rawData)

#### Plot the measured data

In [ ]:
# Plot channel 1 and 2 voltages with respect to time
plt.figure()
#plt.plot(waveform.time(1) * 1e9, waveform.ch(1))
plt.plot(waveform.time(2) * 1e9, waveform.ch(2)*1000/50)
plt.xlabel("Time (ns)")
plt.ylabel("Current (mA)")
plt.grid()
#filename='d01_08.0V'
plt.savefig('d06_0'+str(ep.amplitude)+'.jpg', dpi=200)

# Save the measurement
sp = SampleParameters('d06_0'+str(ep.amplitude))
np.savez_compressed(f"{sp.name}", **waveform.asdict()) #save multiple arrays

#### Load and plot the saved data

In [ ]:
data = np.load(f"{sp.name}"+".npz")
print( data.files)

In [ ]:
plt.figure()
#plt.plot(data['time1'] * 1e9, data['ch1'])
plt.plot(data['time2'] * 1e9, data['ch2'])
plt.xlabel("Time (ns)")
plt.ylabel("Voltage (V)")
plt.grid()
#plt.savefig('filename.jpg')